# Feed Dataset through Queue to Tensorflow from HDFS

## Populate HDFS with Sample Dataset

In [ ]:
%%bash

hadoop fs -copyFromLocal /root/datasets/linear /

In [ ]:
%%bash

hadoop fs -ls /linear

## Create Queue and Feed Tensorflow from HDFS

### Show TensorFlow-HDFS Code

In [ ]:
%%bash

cat /root/src/main/python/queue/tensorflow_hdfs.py

## Open a Terminal through Jupyter Notebook 
### (Menu Bar -> Terminal -> New Terminal)
![Jupyter Terminal](https://s3.amazonaws.com/fluxcapacitor.com/img/jupyter-terminal.png)

## Run the following in the Terminal
```
queue_hdfs
```
_Note:  This will error out in the notebook.  You must run from Terminal!_

In [11]:
%%bash

cat /root/scripts/queue_hdfs

#!/bin/bash

echo "Verifying the dataset in HDFS..."
hadoop fs -ls /linear

echo "Processing dataset from HDFS with TensorFlow..."
python /root/src/main/python/queue/tensorflow_hdfs.py


In [12]:
%%bash

cat /root/src/main/python/queue/tensorflow_hdfs.py 

import tensorflow as tf

filename_queue = tf.train.string_input_producer([
  "hdfs://127.0.0.1:39000/linear/train.csv",
  "hdfs://127.0.0.1:39000/linear/test.csv",
])

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

x_observed, y_pred = tf.decode_csv(value, [[0.0],[0.0]])

with tf.Session() as sess:
  # Start populating the filename queue.
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(coord=coord)
  try:
    for i in range(20):
      # Retrieve and print a single instance:
      example, label = sess.run([x_observed, y_pred])
      print(example, label)
  except tf.errors.OutOfRangeError:
    print("Done!")
  finally:
    coord.request_stop()
    coord.join(threads)
